In [1]:
# Print the current date and time in the format:
# "YYYY-MM-DD HH:MM:SS.microseconds"
import datetime
def print_current_datetime():
    datetime_string = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
    print("CDT(Current Date and Time):", datetime_string)

print_current_datetime()

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

model.save_pretrained("DialoGPT-medium-by-Microsoft")

print_current_datetime()

model.from_pretrained("DialoGPT-medium-by-Microsoft")

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

print_current_datetime()

CDT(Current Date and Time): 2023-03-28 12:42:31.414222
CDT(Current Date and Time): 2023-03-28 12:42:56.851138
>> User:Hi


2023-03-28 12:43:17.231155: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-28 12:43:17.231185: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/gbike/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


DialoGPT: Hi! :D
>> User:Do you speak Korean?
DialoGPT: I do!
>> User:굉장해요.
DialoGPT: I'm Korean, but I don't speak Korean.
>> User:I see.
DialoGPT: I see what you did there.
>> User:What do you mean?
DialoGPT: I see what you did there.
CDT(Current Date and Time): 2023-03-28 12:43:53.472780


In [4]:
# Print the current date and time in the format:
# "YYYY-MM-DD HH:MM:SS.microseconds"
import datetime
def print_current_datetime():
    datetime_string = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
    print("CDT(Current Date and Time):", datetime_string)

print_current_datetime()

# DialoGPT is based on GPT2
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForCausalLM

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = TFAutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Prepare the data
train_data = ["Hello, how are you?", "I am doing well, thanks. How about you?"]
max_len = 512  # Maximum length of input/output sequence
train_encodings = tokenizer(train_data, padding=True, truncation=True, max_length=max_len)

print_current_datetime()

# Define the training settings
epochs = 5
batch_size = 2
learning_rate = 1e-5

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Define the training function
@tf.function
def train_step(inputs):
    with tf.GradientTape() as tape:
        # Get the model output
        outputs = model(inputs["input_ids"], training=True)
        logits = outputs.logits[:, :-1, :]  # Ignore the last token in the sequence

        # Get the loss
        target_ids = inputs["input_ids"][:, 1:]  # Ignore the first token in the sequence
        loss = loss_fn(tf.reshape(target_ids, [-1]), tf.reshape(logits, [-1, model.config.vocab_size]))

    # Update the model
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss

print_current_datetime()

# Train the model
for epoch in range(epochs):
    print(f"Epoch {epoch+1}")
    epoch_loss = 0.0
    for i in range(0, len(train_data), batch_size):
        inputs = {k: v[i:i+batch_size] for k, v in train_encodings.items()}
        batch_loss = train_step(inputs)
        epoch_loss += batch_loss

    print(f"Loss: {epoch_loss.numpy() / (len(train_data) / batch_size):.4f}")

print_current_datetime()

# Save the fine-tuned model
model.save_pretrained("finetuned_DialoGPT-medium")
tokenizer.save_pretrained("finetuned_DialoGPT-medium")

CDT(Current Date and Time): 2023-03-28 14:01:17.265726


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at microsoft/DialoGPT-medium.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Using pad_token, but it is not set yet.


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.